# Data Cleaning on Real Estate Data - Removing Outliers

In this notebook we'll create a function to identify and remove outliers from a dataset based on the value of a column. To identify the outliers, we'll use the interquartile range rule.

Let's start by loading the libraries we'll be using and our dataset.

In [6]:
#Load libraries
import pandas as pd
import numpy as np

In [3]:
#Load dataset
url = 'https://raw.githubusercontent.com/omartinez182/web-apps/master/Scrape_2020.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head()

,revision,price_usd,zona,created_at_first,created_at,days_old,number_of_bedrooms,number_of_bathrooms,m2,latitude,longitude
0,7,350000,Zona 10,5/28/20,5/28/20,0.0,2.0,2.0,158.00,14.595,-90.504
1,4,103896,Zona 18,5/28/20,5/28/20,0.0,5.0,3.0,150.00,14.665,-90.471
2,6,84299,Zona 7,5/28/20,5/28/20,0.0,3.0,2.0,85.00,14.639,-90.560
3,9,90273,Zona 1,5/28/20,5/28/20,0.0,2.0,2.0,69.00,14.646,-90.503
4,13,1375000,Zona 15,2/13/20,5/28/20,105.0,5.0,7.0,664.00,14.612,-90.493


Each observation in the dataset represents a property, and the dataset contains the following attributes for each property:

In [7]:
df.columns

Index(['Unnamed: 0', 'revision', 'price_usd', 'zona', 'created_at_first',
       'created_at', 'days_old', 'number_of_bedrooms', 'number_of_bathrooms',
       'm2', 'latitude', 'longitude'],
      dtype='object')

As expected, the column we'll use to identify and remove outliers is the <b>price_usd</b>, which indicates the price of each property.

The process we'll follow goes as follows:

<ul>
    <li>1. First we need to calculate the interquartile range for the <b>price_usd</b> column.</li>
    <li>2. Then we'll proceed to multiplying the interquartile range (IQR) by 1.5 (constant).</li>
    <li>3. Subsequently, we have to add 1.5 x (IQR) to the 3rd quartile. Any value greater than this can be considered an outlier.</li>
    <li>4. Finally, we have to sbtract 1.5 x (IQR) from the first quartile. Any value smaller than this can be considered an outlier.</li>
</ul>

Before proceeding, let's evaluate the minimum and maximum values for the feature that we'd like to use to remove outliers. 

In [47]:
#Print min & max price values and the shape of the dataframe
print("Min price in USD : $", df['price_usd'].min())
print("Max price in USD : $", df['price_usd'].max())
print("Shape of the dataframe :", df.shape)

Min price in USD : $ 682
Max price in USD : $ 51948052
Shape of the dataframe : (5222, 11)


In order to remove the outliers, first we need to find the interquartile range (IQR), which is defined by the Q1 and Q3. We can easily access quartiles with the <b>"describe( )"</b> method. So let's build a function to indentify and remove any outliers. 

In [37]:
def remove_outliers(df, column):
    "This function takes in a dataframe and removes the outliers using the values of the specified column"
    #Use the describe() method to identify the statistics of interest
    describe = df[column].describe()

    #Create a dictionary for each of the values from the column of interest
    describe_dict = {"count":0,"mean":1,"std":2,"min":3,"25":4,"50":5,"75":6,"max":7}

    #Extract quartiles (Q1, Q3)
    Q1 = describe[describe_dict['25']]
    Q3 = describe[describe_dict['75']]

    #Caculate IQR
    IQR = Q3-Q1

    #Define bounds
    lb = Q1-1.5*IQR
    ub = Q3+1.5*IQR
    print("(IQR = {})A point outside of the following range can be considered an outlier: ({},{})".format(IQR,lb,ub))

    calc_df = df[(df[column] < lb) | (df[column] > ub)]
    print("The number of outliers that will be removed out of {} observations are {}.".format(df[column].size,len(calc_df[column])))

    #remove the outliers from the dataframe
    no_outliers = df[~df[column].isin(calc_df[column])]
    return no_outliers

Now that we have defined our function we can use it to remove the outliers of our dataframe.

In [43]:
#Use function to remove outliers
new_df = remove_outliers(df, 'price_usd')

#Print new min & max price values
print("Min price in USD : $", new_df['price_usd'].min())
print("Max price in USD : $", new_df['price_usd'].max())
print("Number of Observations :", new_df.shape)

(IQR = 195000.0)A point outside of the following range can be considered an outlier: (-177500.0,602500.0)
The number of outliers that will be removed out of 5222 observations are 412.
Min price in USD : $ 682
Max price in USD : $ 600000
Number of Observations : (4810, 11)


As we can see we still have houses that have an unrealistically low price, which weren't singled out with the IQR due to the distribution of our data (skewed). So let's set an arbitrary number to have a minimum price allowed, for this we'll be using our intution and consider the goal of our analysis.

In [45]:
#Set a minimum sale price of $25,000
new_df = new_df[new_df['price_usd'] > 25000]
new_df.shape

(4721, 11)

In [46]:
new_df.head()

,revision,price_usd,zona,created_at_first,created_at,days_old,number_of_bedrooms,number_of_bathrooms,m2,latitude,longitude
0,7,350000,Zona 10,5/28/20,5/28/20,0.0,2.0,2.0,158.00,14.595,-90.504
1,4,103896,Zona 18,5/28/20,5/28/20,0.0,5.0,3.0,150.00,14.665,-90.471
2,6,84299,Zona 7,5/28/20,5/28/20,0.0,3.0,2.0,85.00,14.639,-90.560
3,9,90273,Zona 1,5/28/20,5/28/20,0.0,2.0,2.0,69.00,14.646,-90.503
5,15,280000,NaN,11/19/19,5/28/20,191.0,3.0,3.0,165.00,14.634,-90.506


Finally, we can store the dataframe as a csv file to analyze later or in any other tool that can take flat files as input (Tableau, PowerBI, R, or another Python notebook).

In [49]:
#Save file to CSV
new_df.to_csv(r'/newfile.csv')